In [ ]:
import scapy.all as scapy
import psutil
import requests

# Function to get the Wi-Fi IP address range
def get_wifi_ip():
    for interface, addrs in psutil.net_if_addrs().items():
        if "Wi-Fi" in interface or "wlan" in interface.lower():
            for addr in addrs:
                if addr.family == 2:  # AF_INET (IPv4)
                    ip_split = addr.address.split('.')
                    # Return IP range based on the Wi-Fi adapter
                    return f"{ip_split[0]}.{ip_split[1]}.{ip_split[2]}.0/24"
    return None

# Perform ARP scan and store results in a list
def scan_network(ip_range):
    answered_list, unanswered_list = scapy.arping(ip_range, verbose=0)

    # Store the details of connected devices
    devices = []
    for sent, received in answered_list:
        device_info = {
            "ip": received.psrc,   # Source IP address of the response
            "mac": received.hwsrc  # MAC address of the device
        }
        devices.append(device_info)

    return devices

# Query the storage of a device via its IP
def get_device_storage(ip):
    try:
        response = requests.get(f"http://{ip}:5000/storage", timeout=2)  # Assuming the Flask API is running on port 5000
        if response.status_code == 200:
            data = response.json()
            return data
    except requests.exceptions.RequestException:
        return None

# Main
wifi_ip_range = get_wifi_ip()

if wifi_ip_range:
    print(f"Scanning Wi-Fi network: {wifi_ip_range}")
    devices = scan_network(wifi_ip_range)
    
    print("Devices connected to the Wi-Fi network and their available storage:")
    for device in devices:
        storage_info = get_device_storage(device['ip'])
        if storage_info:
            print(f"IP: {device['ip']}, MAC: {device['mac']}, Free Space: {storage_info['free_space'] / (1024 * 1024 * 1024):.2f} GB")
        else:
            print(f"IP: {device['ip']}, MAC: {device['mac']}, Free Space: Not Available")
else:
    print("Wi-Fi network not found or no IP address assigned.")
